In [9]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import docker
import logging
import time
import concurrent.futures
from datetime import datetime

In [10]:
# Discover the hardware architecture.
avail_cores = os.cpu_count()
print(f"Available cores: {avail_cores}")

Available cores: 48


In [ ]:
# Run docker-activity to monitor power consumption and runtime of the containers.

# Initialize Docker client
client = docker.from_env()
activity_container = client.containers.run(
    image="jdrouet/docker-activity",
    command=["stdout"],
    volumes={
        "/sys/class/powercap": {"bind": "/sys/class/powercap", "mode": "ro"},
        "/var/run/docker.sock": {"bind": "/var/run/docker.sock", "mode": "rw"},
    },
    privileged=True,
    detach=True,
    auto_remove=True
)
for log in activity_container.logs(stream=True, follow=True):
    print(log.decode().strip())

In [10]:
# Run isolated benchmarks in a Docker container on the different cores.
def parse_start_time(start_time_str):
    # Trim to microseconds and remove trailing 'Z'
    if '.' in start_time_str:
        time_part, rest = start_time_str.split('.')
        microseconds = rest[:6]  
        return datetime.strptime(f"{time_part}.{microseconds}", "%Y-%m-%dT%H:%M:%S.%f")
    return datetime.strptime(start_time_str.replace('Z', ''), "%Y-%m-%dT%H:%M:%S")

def parse_die_time(die_time_str):
    if '.' in die_time_str:
        time_part, rest = die_time_str.split('.')
        microseconds = rest[:6]  
        return datetime.strptime(f"{time_part}.{microseconds}", "%Y-%m-%dT%H:%M:%S.%f")
    return datetime.strptime(die_time_str.replace('Z', ''), "%Y-%m-%dT%H:%M:%S")

def run_container(task):
    container = task['client'].containers.run(
        image=task['image'],
        command=task['command'],
        cpuset_cpus=task['cpuset_cpus'],
        cgroupns="private",
        detach=True,
        labels={"test": next((arg.split('=')[1] for arg in task['command'] if arg.startswith('--test=')), None)}
    )
    max_retries = 5
    retry_interval = 1
    start_time = None  
    
    for attempt in range(max_retries):
        container.reload()
        # Capture container metadata
        if container.status == 'running':
            started_at = container.attrs['State']['StartedAt']
            start_time = parse_start_time(started_at)
            print(f"Container for benchmark {task['command']} started successfully.")
            break
        else:
            print(f"Attempt {attempt + 1} failed, retrying in {retry_interval} seconds...")
            time.sleep(retry_interval)
    
    # Ensure start_time is set even if the container does not reach 'running'
    if start_time is None:
        started_at = container.attrs['State']['StartedAt']
        start_time = parse_start_time(started_at)
    
    container.stop()
    container.reload()
    died_at = container.attrs['State']['FinishedAt']
    die_time = parse_die_time(died_at)
    container_lifetime = (die_time - start_time).total_seconds()
    print(f"Container lifetime: {container_lifetime} seconds")
    isolated_benchmarking_results[container.name] = {
        # 'workload': container.attrs['Config']['Labels'],
        'coloc_pair': None,
        'workload': container.attrs['Config']['Labels'].get('test', None), 
        'id': container.id,
        'life_time': container_lifetime,
    }
    container.remove()
    print(f"Container on CPU {task['cpuset_cpus']} completed and removed.")

if __name__ == "__main__":
            
    # Initialize Docker client
    client = docker.from_env()
    
    # Prepare affinity score map for colocated pairs.
    isolated_benchmarking_results = {}

    # Init the disk benchmark.
    print("Container started for Disk benchmark (prepare).")
    disk_prepare_output = client.containers.run(
    image="niklas/sysbench",
    command=[
        "sysbench", "--test=fileio",
        "--file-total-size=50G",
        "--file-test-mode=rndrw",
        "--num-threads=1",
        "prepare"
    ],
    cpuset_cpus="1",
    cgroupns="private",
    detach=False, 
    auto_remove=True  
    )

    # List of dictionaries to hold task information.
    tasks = [
        {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=cpu", "--num-threads=8", "--cpu-max-prime=800000000000","run"], "cpuset_cpus": "1"},
        {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=memory", "--memory-block-size=1M", "--memory-total-size=10G", "--threads=1", "run"], "cpuset_cpus": "2"},
        {"client": client,"image": "niklas/sysbench","command": ["sysbench", "--test=fileio","--file-total-size=50G","--file-test-mode=rndrw","--init-rng=on","--max-time=300","--max-requests=0","run"],"cpuset_cpus": "3"}
    ]
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=2, thread_name_prefix="Isolator") as executor:
        futures = [executor.submit(run_container, task) for task in tasks]
    for future in futures:
        future.result()
    
    # Disk cleanup
    disk_cleanup_output = client.containers.run(
        image="niklas/sysbench",
        command=[
            "sysbench", "--test=fileio",
            "cleanup"
        ],
        cpuset_cpus="3",
        detach=False,  
        auto_remove=True  
    )
    print("Disk benchmark completed.")
        
    print("All isolated benchmarks completed.")
    # Clean up Docker client
    client.close()  

Container started for Disk benchmark (prepare).
Container for benchmark ['sysbench', '--test=cpu', '--num-threads=8', '--cpu-max-prime=800000000000', 'run'] started successfully.
Container for benchmark ['sysbench', '--test=memory', '--memory-block-size=1M', '--memory-total-size=10G', '--threads=1', 'run'] started successfully.
Container lifetime: 0.256175 seconds
Container on CPU 1 completed and removed.
Container lifetime: 9.525112 seconds
Container on CPU 2 completed and removed.
Container for benchmark ['sysbench', '--test=fileio', '--file-total-size=50G', '--file-test-mode=rndrw', '--init-rng=on', '--max-time=300', '--max-requests=0', 'run'] started successfully.
Container lifetime: 0.227651 seconds
Container on CPU 3 completed and removed.
Disk benchmark completed.
All isolated benchmarks completed.


In [11]:
# Access the watched benchmark containers for runtime and power consumption.
print("Container names and IDs:")
for name, container_id in isolated_benchmarking_results.items():
    print(f"{name}: {container_id}")

Container names and IDs:
hungry_mahavira: {'coloc_pair': None, 'workload': 'cpu', 'id': 'f4d2304425e3a9cb86bdecb22b8f8f005933a3209beb71c7001b0855cfeada77', 'life_time': 0.256175}
interesting_bartik: {'coloc_pair': None, 'workload': 'memory', 'id': 'ee058ac616efc31a2145807d1a46228e7a4b10c9a4394cb0866100d7297b4796', 'life_time': 9.525112}
stupefied_ellis: {'coloc_pair': None, 'workload': 'fileio', 'id': '3add30d82a66d55f2c27336fcce05184eb451679402dd68b94240be5af58c6b8', 'life_time': 0.227651}


In [26]:
# Run co-located benchmarks in a Docker container on the same core.
def parse_start_time(start_time_str):
    # Trim to microseconds and remove trailing 'Z'
    if '.' in start_time_str:
        time_part, rest = start_time_str.split('.')
        microseconds = rest[:6]  
        return datetime.strptime(f"{time_part}.{microseconds}", "%Y-%m-%dT%H:%M:%S.%f")
    return datetime.strptime(start_time_str.replace('Z', ''), "%Y-%m-%dT%H:%M:%S")

def parse_die_time(die_time_str):
    if '.' in die_time_str:
        time_part, rest = die_time_str.split('.')
        microseconds = rest[:6]  
        return datetime.strptime(f"{time_part}.{microseconds}", "%Y-%m-%dT%H:%M:%S.%f")
    return datetime.strptime(die_time_str.replace('Z', ''), "%Y-%m-%dT%H:%M:%S")
    
def run_container(task):
    container = task['client'].containers.run(
        image=task['image'],
        command=task['command'],
        cpuset_cpus=task['cpuset_cpus'],
        cgroupns="private",
        detach=True,
        labels={"test": next((arg.split('=')[1] for arg in task['command'] if arg.startswith('--test=')), None)}
    )
    
    max_retries = 5
    retry_interval = 1
    start_time = None  
    
    for attempt in range(max_retries):
        container.reload()
        # Capture container metadata
        if container.status == 'running':
            started_at = container.attrs['State']['StartedAt']
            start_time = parse_start_time(started_at)
            print(f"Container for benchmark {task['command']} started successfully.")
            break
        else:
            print(f"Attempt {attempt + 1} failed, retrying in {retry_interval} seconds...")
            time.sleep(retry_interval)
    
    if start_time is None:
        started_at = container.attrs['State']['StartedAt']
        start_time = parse_start_time(started_at)
    
    container.stop()
    container.reload()
    died_at = container.attrs['State']['FinishedAt']
    die_time = parse_die_time(died_at)
    container_lifetime = (die_time - start_time).total_seconds()
    print(f"Container lifetime: {container_lifetime} seconds")
    coloc_benchmarking_results[container.name] = {
        'coloc_pair': pair_name,
        'workload': container.attrs['Config']['Labels'].get('test', None),  
        'id': container.id,
        'life_time': container_lifetime,
    }
    container.remove()
    print(f"Container on CPU {task['cpuset_cpus']} completed and removed.")

if __name__ == "__main__":
    
    # Initialize Docker client
    client = docker.from_env()
    
    # Prepare affinity score map for colocated pairs.
    coloc_benchmarking_results = {}

    # Init the disk benchmark.
    print("Container started for Disk benchmark (prepare).")
    disk_prepare_output = client.containers.run(
    image="niklas/sysbench",
    command=[
        "sysbench", "--test=fileio",
        "--file-total-size=300G",
        "--file-test-mode=rndrw",
        "--num-threads=1",
        "prepare"
    ],
    cpuset_cpus="1",
    cgroupns="private",
    detach=False, 
    auto_remove=True  
    )

    # Run every co-located benchmark combination on the same core.
    colocation = [
        {"CpuMem": [
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=cpu", "--num-threads=8", "--cpu-max-prime=40000000000", "run"], "cpuset_cpus": "0"},
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=memory", "--memory-block-size=8M", "--memory-total-size=150G", "--threads=1", "run"], "cpuset_cpus": "24"}
        ]},
        {"MemFileIO": [
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=memory", "--memory-block-size=8M", "--memory-total-size=150G", "--threads=1", "run"], "cpuset_cpus": "1"},
            {"client": client,"image": "niklas/sysbench","command": ["sysbench", "--test=fileio","--file-total-size=150G","--file-test-mode=rndrw","--init-rng=on","--max-time=300","--max-requests=0","run"],"cpuset_cpus": "25"}
        ]},
        {"FileIOCpu": [
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=cpu", "--num-threads=8", "--cpu-max-prime=40000000000", "run"], "cpuset_cpus": "2"},
            {"client": client,"image": "niklas/sysbench","command": ["sysbench", "--test=fileio","--file-total-size=300G","--file-test-mode=rndrw","--init-rng=on","--max-time=1800","--max-requests=0","run"],"cpuset_cpus": "26"}
        ]},
        {"CpuCpu": [
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=cpu", "--num-threads=8", "--cpu-max-prime=40000000000", "run"], "cpuset_cpus": "3"},
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=cpu", "--num-threads=8", "--cpu-max-prime=40000000000", "run"], "cpuset_cpus": "27"},
        ]},
        {"MemMem": [
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=memory", "--memory-block-size=8M", "--memory-total-size=150G", "--threads=1", "run"], "cpuset_cpus": "4"},
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=memory", "--memory-block-size=8M", "--memory-total-size=150G", "--threads=1", "run"], "cpuset_cpus": "28"},
        ]},
        {"FileIOFileIO": [
            {"client": client,"image": "niklas/sysbench","command": ["sysbench", "--test=fileio","--file-total-size=300G","--file-test-mode=rndrw","--init-rng=on","--max-time=1800","--max-requests=0","run"],"cpuset_cpus": "5"},
            {"client": client,"image": "niklas/sysbench","command": ["sysbench", "--test=fileio","--file-total-size=300G","--file-test-mode=rndrw","--init-rng=on","--max-time=1800","--max-requests=0","run"],"cpuset_cpus": "29"}
        ]}
    ]
    
    for coloc in colocation:
            for pair_name, tasks in coloc.items():
                print(f"Running colocated tasks for: {pair_name}")
                with concurrent.futures.ThreadPoolExecutor(max_workers=2, thread_name_prefix="Colocator") as executor:
                    futures = [executor.submit(run_container, task) for task in tasks]
                for future in futures:
                    future.result()
                print(f"Completed colocated tasks for: {pair_name}")
    
    # Disk cleanup
    disk_cleanup_output = client.containers.run(
        image="niklas/sysbench",
        command=[
            "sysbench", "--test=fileio",
            "cleanup"
        ],
        cpuset_cpus="3",
        detach=False,  
        auto_remove=True  
    )
    print("Disk benchmark completed.")
        
    print("All colocated benchmarks completed.")
    # Clean up Docker client
    client.close() 

Container started for Disk benchmark (prepare).
Running colocated tasks for: CpuMem
Container for benchmark ['sysbench', '--test=cpu', '--num-threads=8', '--cpu-max-prime=40000000000', 'run'] started successfully.
Container for benchmark ['sysbench', '--test=memory', '--memory-block-size=8M', '--memory-total-size=150G', '--threads=1', 'run'] started successfully.
Container lifetime: 0.1494 seconds
Container on CPU 0 completed and removed.
Container lifetime: 6.346498 seconds
Container on CPU 24 completed and removed.
Completed colocated tasks for: CpuMem
Running colocated tasks for: MemFileIO
Container for benchmark ['sysbench', '--test=memory', '--memory-block-size=8M', '--memory-total-size=150G', '--threads=1', 'run'] started successfully.
Container for benchmark ['sysbench', '--test=fileio', '--file-total-size=150G', '--file-test-mode=rndrw', '--init-rng=on', '--max-time=300', '--max-requests=0', 'run'] started successfully.
Container lifetime: 0.480374 seconds
Container on CPU 25 c

In [27]:
# Access the watched benchmark containers for runtime and power consumption.
print("Container names and IDs:")
for name, container_id in coloc_benchmarking_results.items():
    print(f"{name}: {container_id}")
# print(benchmarking_results)

Container names and IDs:
agitated_elbakyan: {'coloc_pair': 'CpuMem', 'workload': 'cpu', 'id': 'c72a535fb87668763bf844e5287d65bf8cf5221a18712cfaad858a68a1293b34', 'life_time': 0.1494}
gifted_saha: {'coloc_pair': 'CpuMem', 'workload': 'memory', 'id': '953eb625da049da7833f542acdaec1a7ae5eed7a30b84c2cf57d0ba341d11cd7', 'life_time': 6.346498}
stoic_darwin: {'coloc_pair': 'MemFileIO', 'workload': 'fileio', 'id': 'bba135b2bc1c480deb21b2db8061009cad653b764a9182e172d9bc8e6cbe52fe', 'life_time': 0.480374}
eager_brahmagupta: {'coloc_pair': 'MemFileIO', 'workload': 'memory', 'id': '247e3bfd56612464a222a5515c4848b21b248c281eb23c098f9cd2e60653765b', 'life_time': 6.123447}
quirky_burnell: {'coloc_pair': 'FileIOCpu', 'workload': 'cpu', 'id': '0fd1339a4db658d304387868f8bdc7e641a9a4187bdd88793b32395256222cee', 'life_time': 0.295754}
serene_yonath: {'coloc_pair': 'FileIOCpu', 'workload': 'fileio', 'id': '63dd962b016b1305d75f5acd2b0f0cd67924b1f8bbd3056d61ce3eaa57b330c3', 'life_time': 0.342283}
happy_wozni

In [28]:
# Calculate relative distances between the isolated and colocated benchmarks and the affinity score.
# for every coloc pair, get the workload type and add the isolated runtime and power consumption to the dict.add()
for name, result in coloc_benchmarking_results.items():
    print(f"Colocated benchmark: {result.get('workload')}, {result.get('coloc_pair')}")
    result['isolated_runtime'] = 'not yet computed'
for name, container_id in coloc_benchmarking_results.items():
    print(f"{name}: {container_id}")

Colocated benchmark: cpu, CpuMem
Colocated benchmark: memory, CpuMem
Colocated benchmark: fileio, MemFileIO
Colocated benchmark: memory, MemFileIO
Colocated benchmark: cpu, FileIOCpu
Colocated benchmark: fileio, FileIOCpu
Colocated benchmark: cpu, CpuCpu
Colocated benchmark: cpu, CpuCpu
Colocated benchmark: memory, MemMem
Colocated benchmark: memory, MemMem
Colocated benchmark: fileio, FileIOFileIO
Colocated benchmark: fileio, FileIOFileIO
agitated_elbakyan: {'coloc_pair': 'CpuMem', 'workload': 'cpu', 'id': 'c72a535fb87668763bf844e5287d65bf8cf5221a18712cfaad858a68a1293b34', 'life_time': 0.1494, 'isolated_runtime': 'not yet computed'}
gifted_saha: {'coloc_pair': 'CpuMem', 'workload': 'memory', 'id': '953eb625da049da7833f542acdaec1a7ae5eed7a30b84c2cf57d0ba341d11cd7', 'life_time': 6.346498, 'isolated_runtime': 'not yet computed'}
stoic_darwin: {'coloc_pair': 'MemFileIO', 'workload': 'fileio', 'id': 'bba135b2bc1c480deb21b2db8061009cad653b764a9182e172d9bc8e6cbe52fe', 'life_time': 0.480374, 

In [ ]:
# Write the resulting affinity score matrix
matrix = pd.DataFrame.from_dict(coloc_benchmarking_results, orient='index')

# Drop the 'id' column and reset the index to remove container names.
# matrix.drop(columns=['id', 'coloc_pair'], inplace=True)
# matrix.reset_index(drop=True, inplace=True)

# Rename columns to match your desired structure.
# matrix.rename(columns={
#     'workload': 'Workload A',
#     'life_time': 'Isolated Runtime 1 (s)'
# }, inplace=True)

# Duplicate columns for 'Workload B' and 'Isolated Runtime 2 (s)'.
# matrix['Workload B'] = matrix['Workload A']
# matrix['Isolated Runtime 2 (s)'] = matrix['Isolated Runtime 1 (s)']

# Add additional columns for other metrics.
matrix['Colocated Runtime 1 (s)'] = 1
matrix['Colocated Runtime 2 (s)'] = 1
matrix['Relative Difference (%)'] = 1
matrix['Average Relative Difference (%)'] = 1
matrix['Affinity Score'] = 1

print(matrix)

Empty DataFrame
Columns: [Colocated Runtime 1 (s), Colocated Runtime 2 (s), Relative Difference (%), Average Relative Difference (%), Affinity Score]
Index: []
